In [ ]:
import sys
import os
sys.path.append(os.path.abspath("..")) 

from dotenv import load_dotenv
load_dotenv()

In [1]:
import os
import json
import pickle

from tqdm.notebook import tqdm
from dotenv import load_dotenv

import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import DataLoader

from src.models import ElixrbClassifier

# HELPER FUNCTIONS

In [2]:


def _prepare_output_dir(output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    output_dir = os.path.join(output_dir, 'elixrb-classifier')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    return output_dir

def _save_model_and_config(model, config:dict = None, output_dir=None):
    if output_dir is None:
        output_dir = 'output'
        
    output_dir = _prepare_output_dir(output_dir)
    
    torch.save(model.state_dict(), os.path.join(output_dir, 'model.pth'))
        
    if config is not None:
        if not isinstance(config, dict):
            raise ValueError("config should be a dictionary")
            
        if os.path.exists(os.path.join(output_dir, 'config.json')):
            os.remove(os.path.join(output_dir, 'config.json'))
            
        with open(os.path.join(output_dir, 'config.json'), 'w') as f:
            json.dump(config, f)

# Trainer and Evaution Code Block

In [3]:



def train_model(model, train_loader, num_epochs=100, learning_rate=0.001, 
                weight_decay=0.01, output_dir=None, device=None):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                    factor=0.5, patience=3, verbose=True)
    
    device = device
    model = model.to(device)
    
    config = {
        "num_epochs": num_epochs,
        "learning_rate": learning_rate,
        "optimizer": optimizer.__class__.__name__,
        "weight_decay": weight_decay,
        "criterion": criterion.__class__.__name__,
        "scheduler": scheduler.__class__.__name__
    }
    
    best_loss = float('inf')
    
    # Ana epoch döngüsü için tqdm
    epoch_pbar = tqdm(range(num_epochs), desc='Training', unit='epoch')
    
    for epoch in epoch_pbar:
        model.train()
        running_loss = 0.0
        
        # Batch döngüsü için tqdm
        batch_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', 
                         leave=False, unit='batch')
        
        for features, labels in batch_pbar:
            features, labels = features.to(device), labels.to(device)
            
            optimizer.zero_grad()
            logits, _ = model(features)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            # Batch progress bar'ı güncelle
            batch_pbar.set_postfix({'batch_loss': f'{loss.item():.4f}'})
        
        epoch_loss = running_loss / len(train_loader)
        scheduler.step(epoch_loss)
        
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            config["best_epoch"] = epoch
            config["best_loss"] = best_loss
            _save_model_and_config(model, config, output_dir)
        
        # Epoch progress bar'ı güncelle
        epoch_pbar.set_postfix({
            'loss': f'{epoch_loss:.4f}',
            'best_loss': f'{best_loss:.4f}',
            'lr': f'{optimizer.param_groups[0]["lr"]:.6f}'
        })
        
        if (epoch + 1) % 5 == 0:
            print(f'\nEpoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
    
    return model, config

def evaluate_model(model, test_loader, device):
    device = device
    model = model.to(device)
    model.eval()
    
    correct = 0
    total = 0
    class_correct = [0] * 14
    class_total = [0] * 14
    
    # Test için tqdm ekle
    test_pbar = tqdm(test_loader, desc='Evaluating', unit='batch')
    
    with torch.no_grad():
        for features, labels in test_pbar:
            features, labels = features.to(device), labels.to(device)
            logits, _ = model(features)
            _, predicted = torch.max(logits.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += (predicted[i] == label).item()
                class_total[label] += 1
            
            # Progress bar'ı güncelle
            current_acc = 100 * correct / total
            test_pbar.set_postfix({'accuracy': f'{current_acc:.2f}%'})
    
    accuracy = 100 * correct / total
    print(f'\nGenel Test Doğruluğu: {accuracy:.2f}%')
    
    print('\nSınıf Bazında Doğruluk:')
    for i in range(14):
        if class_total[i] > 0:
            class_acc = 100 * class_correct[i] / class_total[i]
            print(f'Sınıf {i}: {class_acc:.2f}%')
    
    return accuracy

In [4]:

load_dotenv()

dataset_path = os.getenv('SMOTE_RESAMPLED_DATASET_PATH')

with open(dataset_path, 'rb') as f:
    data = pickle.load(f)


embeddings = data['X_resampled']
labels = data['y_resampled']
mlb = data['mlb']

In [5]:
print(f'embeddings shape: {embeddings.shape}')
print(f'labels shape: {labels.shape}')

print(f'labels: {mlb}')

print(f'Number of classes: {len(mlb)}')

print("sample binarized labels: ", labels[0])

print("sample embeddings: ", embeddings[0])


embeddings shape: (268980, 32, 128)
labels shape: (268980, 15)
labels: ['Atelectasis' 'Cardiomegaly' 'Consolidation' 'Edema' 'Effusion'
 'Emphysema' 'Fibrosis' 'Hernia' 'Infiltration' 'Mass' 'No Finding'
 'Nodule' 'Pleural_Thickening' 'Pneumonia' 'Pneumothorax']
Number of classes: 15
sample binarized labels:  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
sample embeddings:  [[-0.13627037 -0.05453796 -0.12541597 ...  0.06783227 -0.117041
  -0.0268871 ]
 [ 0.05250404  0.0477107  -0.06727851 ...  0.0846091  -0.06936242
  -0.01200239]
 [-0.1040024   0.02110123 -0.01501682 ...  0.16914235 -0.15074338
   0.04070614]
 ...
 [ 0.1983867   0.1267138   0.04158648 ... -0.16025399  0.06184255
   0.04785817]
 [ 0.06910649  0.0430859  -0.06436938 ...  0.07458547 -0.03532307
  -0.00727787]
 [ 0.07154053  0.19609863  0.02250194 ... -0.00234656  0.08438659
   0.09025895]]


In [6]:
train_val_ratio = 0.8
batch_size = 512
epochs = 200
learning_rate = 0.003
weight_decay = 0.01
device = 'cuda:2' if torch.cuda.is_available() else 'cpu'



In [7]:
class ElixrbDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

train_val_size = int(len(embeddings) * train_val_ratio)

train_features = embeddings[:train_val_size]

train_labels = labels[:train_val_size]

val_features = embeddings[train_val_size:]

val_labels = labels[train_val_size:]



train_dataset = ElixrbDataset(train_features, train_labels)

val_dataset = ElixrbDataset(val_features, val_labels)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)



In [8]:
clip_embed_size = 768
input_size = 4096
num_classes = len(mlb)

model =  ElixrbClassifier(
        input_size=input_size,
        embedding_dim=clip_embed_size,
        num_classes=num_classes)


if os.path.exists('./weights/elixrb-classifier/model.pth'):
    model.load_state_dict(torch.load('./weights/elixrb-classifier/model.pth'))


model, config = train_model(model, train_loader, num_epochs=epochs,learning_rate=learning_rate,weight_decay=weight_decay,output_dir='./weights',device=device)

/home/yasin/Projects/elixr-diffusion/.venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Training:   0%|          | 0/200 [00:00<?, ?epoch/s]

Epoch 1/200:   0%|          | 0/421 [00:00<?, ?batch/s]

Epoch 2/200:   0%|          | 0/421 [00:00<?, ?batch/s]

Epoch 3/200:   0%|          | 0/421 [00:00<?, ?batch/s]

...

Epoch 198/200:   0%|          | 0/421 [00:00<?, ?batch/s]

Epoch 199/200:   0%|          | 0/421 [00:00<?, ?batch/s]

Epoch 200/200:   0%|          | 0/421 [00:00<?, ?batch/s]


Epoch [200/200], Loss: 0.9959


: 